In [1]:
import os
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import time
import csv

# Mostrar el directorio actual
print(os.getcwd())

# Cambiar el directorio de trabajo
os.chdir('/Users/matdknu/Dropbox/social-data-science/wiki-chile_project')



/Users/matdknu/Dropbox/social-data-science/wiki-chile_project/scripts/scrapping


In [2]:

# Leer el archivo original con la lista de personas a scrapear
df_wikipedia = pd.read_excel('data/raw_data/familia_link_manual2.xlsx')


# Mostrar las primeras filas de ambos DataFrames
print("Wikipedia:")
print(df_wikipedia.head())



Wikipedia:
          Familia                                                URL
0  Famlia Allende        https://es.wikipedia.org/wiki/Ramón_Allende
1  Famlia Allende  https://es.wikipedia.org/wiki/Salvador_Allende...
2  Famlia Allende  https://es.wikipedia.org/wiki/Laura_Allende_Go...
3  Famlia Allende     https://es.wikipedia.org/wiki/Salvador_Allende
4  Famlia Allende  https://es.wikipedia.org/wiki/Andrés_Pascal_Al...


In [3]:
# Normalizar nombres de columnas
df_wikipedia.columns = df_wikipedia.columns.str.strip().str.lower()

# Extraer links únicos (suponiendo que la columna se llama 'link')
initial_urls = df_wikipedia['url'].dropna().unique().tolist()

# Mostrar resultado formateado para pegar
print("initial_urls = [")
for url in initial_urls:
    print(f'    "{url}",')
print("]")


initial_urls = [
    "https://es.wikipedia.org/wiki/Ramón_Allende",
    "https://es.wikipedia.org/wiki/Salvador_Allende_Castro",
    "https://es.wikipedia.org/wiki/Laura_Allende_Gossens",
    "https://es.wikipedia.org/wiki/Salvador_Allende",
    "https://es.wikipedia.org/wiki/Andrés_Pascal_Allende",
    "https://es.wikipedia.org/wiki/Denise_Pascal",
    "https://es.wikipedia.org/wiki/Hortensia_Bussi",
    "https://es.wikipedia.org/wiki/Beatriz_Allende",
    "https://es.wikipedia.org/wiki/Miria_Contreras",
    "https://es.wikipedia.org/wiki/Maya_Fernández",
    "https://es.wikipedia.org/wiki/Isabel_Allende_Bussi",
    "https://es.wikipedia.org/wiki/Carmen_Castillo_(cineasta)",
    "https://es.wikipedia.org/wiki/Isabel_Allende",
    "https://es.wikipedia.org/wiki/Paula_Frías_Allende",
    "https://es.wikipedia.org/wiki/Marcia_Tambutti",
    "https://es.wikipedia.org/wiki/José_Miguel_Carrera",
    "https://es.wikipedia.org/wiki/Luis_Aldunate",
    "https://es.wikipedia.org/wiki/Ignacio_de

In [4]:
CSV_FILENAME = "datos_chile_manual2.csv"
BASE_URL = "https://es.wikipedia.org"


visited_urls = set()

# Función para extraer información de una persona o entidad
def extract_info(url, depth=0, max_depth=2):
    if url in visited_urls or depth > max_depth:
        return None
    visited_urls.add(url)

    response = requests.get(url)
    if response.status_code != 200:
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    infobox = soup.find("table", class_="infobox biography vcard")
    if not infobox:
        return None

    info = {
        "Nombre": "",
        "Fecha de nacimiento": "",
        "Residencia": "",
        "Nacionalidad": "",
        "Cargos": "",
        "Educación": "",
        "Trabajos previos": "",
        "Área": "",
        "Partido político": "",
        "Familia": "",
        "URL": url
    }

    title = soup.find("h1", class_="firstHeading")
    if title:
        info["Nombre"] = title.text.strip()

    for row in infobox.find_all("tr"):
        header = row.find("th")
        value = row.find("td")
        if header and value:
            header_text = header.text.strip()
            value_text = value.text.strip()

            if "Nacimiento" in header_text:
                info["Fecha de nacimiento"] = value_text
            elif "Residencia" in header_text:
                info["Residencia"] = value_text
            elif "Nacionalidad" in header_text:
                info["Nacionalidad"] = value_text
            elif any(k in header_text for k in ["Ocupación", "Cargos", "Cargo"]):
                for item in value.stripped_strings:
                    info["Cargos"] += item + "; "
            elif any(k in header_text for k in ["Educación", "Educado en", "Educada en"]):
                education_links = [link.text for link in value.find_all("a") if link.get("href", "").startswith("/wiki/")]
                info["Educación"] = "; ".join(education_links)
            elif "Área" in header_text:
                info["Área"] = value_text
            elif "Partido político" in header_text:
                parties = [link.text for link in value.find_all("a")]
                info["Partido político"] = "; ".join(parties)
            elif any(k in header_text for k in ["Padres", "Cónyuge", "Hijos", "Familia"]):
                family_links = []
                for link in value.find_all("a"):
                    href = link.get("href", "")
                    if (href.startswith("/wiki/") and
                        not any(href.startswith(excl) for excl in [
                            "/wiki/Ayuda:", "/wiki/Archivo:", "/wiki/Especial:", "/wiki/Plantilla:", "/wiki/Portal:", "/wiki/Categor%C3%ADa:", "/wiki/Familia_"])):
                        family_links.append(f"{link.text} ({BASE_URL}{href})")
                    else:
                        family_links.append(link.text)
                info["Familia"] += "; ".join(family_links) + "; "

    # Extraer trabajos previos desde bloques válidos únicamente
    rows = infobox.find_all("tr")
    for i in range(len(rows) - 1):
        th = rows[i].find("th")
        td = rows[i + 1].find("td")
        if th and td and th.get("colspan") == "3" and 'background-color:#E6E6FA' in th.get("style", ""):
            if th.find("a"):  # asegurarse que tiene enlace, para evitar otros bloques como altura, religión, etc.
                cargos = [a.text for a in th.find_all("a") if a.get("href", "").startswith("/wiki/")]
                fechas = td.get_text(strip=True)
                cargo_str = "; ".join(cargos)
                info["Trabajos previos"] += f"{cargo_str} – {fechas}; "

    save_to_csv(info)
    print(f"Guardado: {info['Nombre']}")

    for family_member in info["Familia"].split("; "):
        if "(" in family_member and ")" in family_member:
            family_url = family_member.split("(")[1].split(")")[0]
            if ("wikipedia" in family_url and not "Familia_" in family_url):
                extract_info(family_url, depth + 1, max_depth)
                time.sleep(2)

# Guardar datos en CSV
def save_to_csv(data):
    with open(CSV_FILENAME, "a", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=data.keys())
        if file.tell() == 0:
            writer.writeheader()
        writer.writerow(data)

# Limpiar archivo CSV al inicio
open(CSV_FILENAME, "w", encoding="utf-8").close()

# Iniciar scraping
for url in initial_urls:
    extract_info(url)
    time.sleep(2)

# Eliminar duplicados
df = pd.read_csv(CSV_FILENAME)
df.drop_duplicates(subset=["Nombre"], keep="first", inplace=True)
df.to_csv(CSV_FILENAME, index=False)
print("Proceso completado. Datos guardados en", CSV_FILENAME)


Guardado: Ramón Allende
Guardado: Salvador Allende Castro
Guardado: Salvador Allende
Guardado: Laura Allende Gossens
Guardado: Isabel Allende Bussi
Guardado: Hortensia Bussi
Guardado: Marcia Tambutti
Guardado: Maya Fernández
Guardado: Beatriz Allende
Guardado: Isabel Allende
Guardado: Ramón Allende
Guardado: Andrés Pascal Allende
Guardado: Pedro Pascal
Guardado: Lux Pascal
Guardado: Javiera Balmaceda
Guardado: Denise Pascal
Guardado: Andrés Pascal Allende
Guardado: Salvador Allende
Guardado: Marcia Tambutti
Guardado: Denise Pascal
Guardado: Isabel Allende
Guardado: Miria Contreras
Guardado: Maya Fernández
Guardado: Carmen Castillo (cineasta)
Guardado: Fernando Castillo Velasco
Guardado: Mónica Echeverría
Guardado: Paula Frías Allende
Guardado: José Miguel Carrera
Guardado: Ignacio de la Carrera y de las Cuevas
Guardado: Ignacio de la Carrera y Ureta
Guardado: Juan José Carrera
Guardado: José Miguel Carrera
Guardado: Luis Carrera
Guardado: Javiera Carrera
Guardado: Ignacio Carrera Pinto